# **Практическая работа №4. Введение в Python. ООП рефакторинг**

---




**Обучающийся:** *Коротеев Владислав Николаевич*  



---

## **Цель работы:**



Провести рефакторинг ранее созданного Python-пакета в предыдущей практической работе, преобразовав его функциональность в соответствии с принципами объектно-ориентированного программирования. Обновленный пакет должен включать классы и методы для преобразования координат между декартовой и сферической системами координат, а также для работы с файлами.



## **Задачи:**



1. Анализ существующего пакета и определение необходимых изменений для перехода на ООП-парадигму.

2. Создание новой структуры пакета с использованием классов и модулей, соответствующих принципам ООП.

3. Реализация классов и методов для преобразования координат между декартовой и сферической системами координат.

4. Реализация классов и методов для работы с файлами, обеспечивая удобный интерфейс для чтения и записи данных.

5. Создание файла `__main__.py` с консольным интерфейсом для взаимодействия с функциональностью пакета.

6. Тестирование и проверка работоспособности обновленного пакета на примерах, подтверждение корректности реализации.



## **Демонстрация результата:**



Вставьте код каждого из ваших модулей в соответствующие ячейки ниже.

**Строку, начинающуюся на %%writefile... стирать запрещено**

### **Содержимое модуля \_\_init__.py:**

In [ ]:
!mkdir geo_transform

In [ ]:
%%writefile geo_transform/__init__.py

# Ваш код
from .transformations import CoordinateConverter
from .file_operations import FileManager
from .utils import AngleConverter
from .consol_interface import ConsoleInterface

__all__ = [
    "CoordinateConverter",
    "FileManager",
    "AngleConverter",
    "ConsoleInterface",
]

### **Содержимое модуля transformations.py:**

In [ ]:
%%writefile geo_transform/transformations.py

# Ваш код
from math import sqrt, atan2, sin, cos
from .utils import AngleConverter


class CoordinateConverter:
    """Класс для преобразования координат между декартовой и сферической системами координат."""

    def __init__(self):
        """Инициализация конвертера координат."""
        self.angle_converter = AngleConverter()

    def cartesian_to_spherical(
        self, x: float, y: float, z: float
    ) -> tuple[float, float, float]:
        """Преобразование декартовых координат в сферические.

        Args:
            x (float): координата x.
            y (float): координата y.
            z (float): координата z.

        Returns:
            tuple[float, float, float]: Кортеж (r, θ, ϕ), где:
                r (float): радиус.
                θ (float): азимутальный угол в градусах.
                ϕ (float): полярный угол в градусах.
        """
        r = sqrt(x**2 + y**2 + z**2)
        teta = 2 * atan2(y, x + sqrt(x**2 + y**2))
        fi = atan2(sqrt(x**2 + y**2), z)

        teta = self.angle_converter.rad_to_deg(teta)
        fi = self.angle_converter.rad_to_deg(fi)

        return r, teta, fi

    def spherical_to_cartesian(
        self, r: float, teta: float, fi: float
    ) -> tuple[float, float, float]:
        """Преобразование сферических координат в декартовые.

        Args:
            r (float): радиус.
            teta (float): азимутальный угол в градусах.
            fi (float): полярный угол в градусах.

        Returns:
            tuple[float, float, float]: Кортеж (x, y, z).
        """
        # Переводим все в радианы
        teta = self.angle_converter.deg_to_rad(teta)
        fi = self.angle_converter.deg_to_rad(fi)

        x = r * sin(fi) * cos(teta)
        y = r * sin(fi) * sin(teta)
        z = r * cos(fi)

        return x, y, z

### **Содержимое модуля utils.py:**

In [ ]:
%%writefile geo_transform/utils.py

# Ваш код
from math import pi


class AngleConverter:
    """Класс для преобразования углов между градусами и радианами."""

    @staticmethod
    def deg_to_rad(degrees: float) -> float:
        """Преобразовать градусы в радианы.

        Args:
            degrees (float): Углы в градусах.

        Returns:
            float: Угол в радианах.
        """
        return degrees * (pi / 180)

    @staticmethod
    def rad_to_deg(radians: float) -> float:
        """Преобразовать радианы в градусы.

        Args:
            radians (float): Угол в радианах.

        Returns:
            float: Угол в градусах.
        """
        return radians * (180 / pi)

### **Содержимое модуля file_operations.py:**

In [ ]:
%%writefile geo_transform/file_operations.py

# Ваш код
from typing import List, Tuple


class FileManager:
    """Класс для работы с файлами для чтения и записи координат."""

    def __init__(self, default_output_file: str = "results.txt"):
        """Инициализация менеджера файлов.

        Args:
            default_output_file (str): Имя файла по умолчанию для записи результатов.
        """
        self.default_output_file = default_output_file

    def write_results_to_file(self, data: str, filename: str = None) -> None:
        """Запись результатов в файл.

        Args:
            data (str): Данные для записи.
            filename (str, optional): Имя файла. Если не указано, используется файл по умолчанию.
        """
        if filename is None:
            filename = self.default_output_file

        try:
            with open(filename, "a", encoding="utf-8") as file:
                file.write(data + "\n")
        except IOError as e:
            print(f"Ошибка при записи в файл '{filename}': {e}")

    def read_coordinates_from_file(
        self, filename: str
    ) -> List[Tuple[float, float, float]]:
        """Чтение координат из файла.

        Args:
            filename (str): Имя файла для чтения.

        Returns:
            List[Tuple[float, float, float]]: Список кортежей координат (x, y, z).
        """
        try:
            with open(filename, "r", encoding="utf-8") as f:
                lines = f.readlines()
        except FileNotFoundError:
            print(f"Ошибка: файл '{filename}' не найден.")
            return []

        coords = []
        for line_num, line in enumerate(lines, start=1):
            parts = line.strip().split()
            if len(parts) == 3:
                try:
                    coords.append(tuple(map(float, parts)))
                except ValueError:
                    print(f"Пропущена строка {line_num} (не числа): {line.strip()}")
            elif line.strip():  # Игнорируем пустые строки
                print(
                    f"Пропущена строка {line_num} (неверное количество значений): {line.strip()}"
                )

        return coords

    def set_default_output_file(self, filename: str) -> None:
        """Установить файл по умолчанию для записи результатов.

        Args:
            filename (str): Имя файла.
        """
        self.default_output_file = filename

### **Содержимое модуля \_\_main__.py:**

In [ ]:
%%writefile geo_transform/__main__.py

# Ваш код
from .consol_interface import ConsoleInterface


def main():
    """Главная функция для запуска консольного интерфейса."""
    interface = ConsoleInterface()
    interface.run()


if __name__ == "__main__":
    main()

### **Содержимое модуля main.py (с импортом пакета и тестированием функций из него):**

In [ ]:
# Ваш код
"""
Тестовый файл для проверки пакета
"""

from geo_transform_class import CoordinateConverter, FileManager, AngleConverter


def test_coordinate_converter():
    """Тестирование класса CoordinateConverter."""
    print("=" * 60)
    print("Тест 1: Преобразование декартовых координат в сферические")
    print("=" * 60)

    converter = CoordinateConverter()

    # Тест 1: Простые координаты
    x, y, z = 1.0, 1.0, 1.0
    r, teta, fi = converter.cartesian_to_spherical(x, y, z)
    print(f"Входные данные (x, y, z): ({x}, {y}, {z})")
    print(f"Результат (r, θ, ϕ): ({r:.6f}, {teta:.6f}, {fi:.6f})")

    # Обратное преобразование для проверки
    x2, y2, z2 = converter.spherical_to_cartesian(r, teta, fi)
    print(f"Обратное преобразование (x, y, z): ({x2:.6f}, {y2:.6f}, {z2:.6f})")
    print()

    # Тест 2: Сферические в декартовые
    print("=" * 60)
    print("Тест 2: Преобразование сферических координат в декартовые")
    print("=" * 60)

    r, teta, fi = 5.0, 45.0, 60.0
    x, y, z = converter.spherical_to_cartesian(r, teta, fi)
    print(f"Входные данные (r, θ, ϕ): ({r}, {teta}, {fi})")
    print(f"Результат (x, y, z): ({x:.6f}, {y:.6f}, {z:.6f})")

    # Обратное преобразование для проверки
    r2, teta2, fi2 = converter.cartesian_to_spherical(x, y, z)
    print(f"Обратное преобразование (r, θ, ϕ): ({r2:.6f}, {teta2:.6f}, {fi2:.6f})")
    print()


def test_file_manager():
    """Тестирование класса FileManager."""
    print("=" * 60)
    print("Тест 3: Работа с файлами")
    print("=" * 60)

    file_manager = FileManager("test_results.txt")

    # Запись данных
    test_data = "Тестовая строка для записи в файл"
    file_manager.write_results_to_file(test_data)
    print(f"Данные записаны в файл: {file_manager.default_output_file}")

    # Создание тестового файла с координатами
    test_input_file = "test_coordinates.txt"
    with open(test_input_file, "w", encoding="utf-8") as f:
        f.write("1.0 2.0 3.0\n")
        f.write("4.0 5.0 6.0\n")
        f.write("10.0 0.0 0.0\n")

    print(f"Создан тестовый файл: {test_input_file}")

    # Чтение координат
    coords = file_manager.read_coordinates_from_file(test_input_file)
    print(f"Прочитано координат: {len(coords)}")
    for idx, coord in enumerate(coords, start=1):
        print(f"  Набор {idx}: {coord}")
    print()


def test_angle_converter():
    """Тестирование класса AngleConverter."""
    print("=" * 60)
    print("Тест 4: Преобразование углов")
    print("=" * 60)

    angle_converter = AngleConverter()

    # Тест градусы -> радианы -> градусы
    degrees = 180.0
    radians = angle_converter.deg_to_rad(degrees)
    degrees_back = angle_converter.rad_to_deg(radians)

    print(f"Градусы: {degrees}")
    print(f"Радианы: {radians:.10f}")
    print(f"Обратно в градусы: {degrees_back:.10f}")
    print()


def main():
    """Главная функция для запуска всех тестов."""
    print("\n" + "=" * 60)
    print("Тестирование пакета")
    print("=" * 60 + "\n")

    try:
        test_coordinate_converter()
        test_file_manager()
        test_angle_converter()

        print("=" * 60)
        print("Все тесты завершены успешно!")
        print("=" * 60)

    except Exception as e:
        print(f"\nОшибка при тестировании: {e}")
        import traceback

        traceback.print_exc()


if __name__ == "__main__":
    main()